In [1]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


Te amo


In [2]:
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
pdf_path = list(uploaded.keys())[0]


Saving 1. SYLLABUS PROGRAMACION BASICA.pdf to 1. SYLLABUS PROGRAMACION BASICA.pdf
User uploaded file "1. SYLLABUS PROGRAMACION BASICA.pdf" with length 336365 bytes


In [3]:
!pip install pdfminer.six


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.9 MB/s eta 0:00:00


In [4]:
!pip install pdfminer.six unidecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.9 MB/s eta 0:00:00


In [5]:
import pdfplumber

def extract_tables_from_pdf():
    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []
        previous_table = None
        for page_num, page in enumerate(pdf.pages, 1):
            tables = page.extract_tables()
            for idx, table in enumerate(tables):
                if previous_table and len(previous_table[0]) == len(table[0]):
                    # Si la tabla anterior tiene el mismo número de columnas que la tabla actual,
                    # se asume que es una continuación y se unen
                    merged_table = previous_table + table
                    all_tables.append(merged_table)
                    print(f"\n===== Tabla Continuada en Página {page_num} =====")
                    for row in merged_table:
                        print(row)
                    previous_table = None  # Resetear la tabla anterior después de fusionar
                else:
                    # Si no es una continuación, se imprime como una nueva tabla
                    print(f"\n===== Tabla {idx + 1} en Página {page_num} =====")
                    for row in table:
                        print(row)
                    all_tables.append(table)
                    previous_table = table
        return all_tables

tables = extract_tables_from_pdf()


ModuleNotFoundError: ignored

In [ ]:
from pdfminer.high_level import extract_text
from unidecode import unidecode
import re
import pdfplumber

def is_uppercase(string):
    return string.isupper()

def extract_section_with_title_and_table(keyword):
    def extract_table_column(table, keyword):
        col_index = None
        for idx, row in enumerate(table):
            uppercase_columns = [i for i, cell in enumerate(row) if is_uppercase(str(cell))]
            if uppercase_columns:
                matching_columns = [i for i in uppercase_columns if keyword in str(row[i])]
                if matching_columns:
                    col_index = matching_columns[0]
                    break
        return col_index

    # Primero, buscaremos la palabra clave en el texto
    text = extract_text(pdf_path)
    lines = text.split('\n')

    section_content = []
    capture_content = False

    for line in lines:
        line_without_accent = unidecode(line)

        if keyword in line_without_accent:
            capture_content = True

        if capture_content:
            section_content.append(line)

        # Si encontramos una línea completamente en mayúsculas después de la keyword, paramos
        if capture_content and line_without_accent.isupper() and keyword not in line_without_accent:
            break

    if section_content:
        return section_content

    # Si no se encontró la palabra clave en el texto, buscar en las tablas
    content_from_table = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                col_index = extract_table_column(table, keyword)
                if col_index is not None:
                    for row in table:
                        if len(row) > col_index:
                            content_from_table.append(row[col_index])

    return content_from_table

# Palabra clave que deseas buscar en el PDF
keyword = "PROGRAMA SINTÉTICO"

# Extraer la sección con la palabra clave y mostrar las salidas
section = extract_section_with_title_and_table(keyword)
for line in section:
    print(line)






In [6]:
from pdfminer.high_level import extract_text
from unidecode import unidecode
import re

def extract_section_with_title(pdf_path):
    text = extract_text(pdf_path)
    lines = text.split('\n')

    # Lista de palabras clave
    keywords = ["PROGRAMA SINTETICO", "UNIDADES", "TEMAS"]

    # Patrón para identificar números romanos seguido de cualquier texto
    roman_pattern = re.compile(r'^(I{1,3}|IV|V?I{0,3}|IX|X{1,3})\.\s+')

    section_content = []
    capture_content = False

    for line in lines:
        line_without_accent = unidecode(line)

        # Ignora líneas vacías (saltos de página)
        if not line.strip():
            continue

        if capture_content:
            roman_match = roman_pattern.match(line_without_accent)
            if roman_match:
                # Extractamos la parte después del número romano
                after_roman = line_without_accent[roman_match.end():].split()
                if after_roman and after_roman[0].isupper():
                    break
            elif any(keyword in line_without_accent for keyword in keywords) or line_without_accent.isupper():
                break

        if not capture_content:
            for keyword in keywords:
                if keyword in line_without_accent:
                    capture_content = True
                    break

        if capture_content:
            section_content.append(line)

    return section_content

section = extract_section_with_title(pdf_path)
for line in section:
    print(line)


PROGRAMA SINTÉTICO 


In [7]:
!pip install pdfminer.six


In [8]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.5 MB/s eta 0:00:00


In [9]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.2 MB/s eta 0:00:00


In [ ]:
import pdfplumber

def is_uppercase(string):
    return string.isupper()

def extract_column_with_uppercase_keyword(pdf_path, keyword):
    with pdfplumber.open(pdf_path) as pdf:
        all_data = []
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                for idx, row in enumerate(table):
                    # Identificar si la fila actual tiene celdas en mayúsculas (potenciales encabezados)
                    uppercase_columns = [i for i, cell in enumerate(row) if is_uppercase(str(cell))]
                    if uppercase_columns:
                        # Verificar si el keyword está en las celdas identificadas
                        matching_columns = [i for i in uppercase_columns if keyword in str(row[i])]
                        if matching_columns:
                            col_index = matching_columns[0]
                            # Recopilar datos de la columna especificada de las filas subsiguientes
                            for subsequent_row in table[idx+1:]:
                                all_data.append(subsequent_row[col_index])
                            break  # Rompemos el ciclo una vez que encontramos el keyword
                #print("Encabezado detectado:", table[0])  # Imprimir encabezado para revisión
        return all_data

# Extraer información de la columna con encabezado en mayúsculas "PROGRAMA SINTÉTICO"
data = extract_column_with_uppercase_keyword(pdf_path, "PROGRAMA SINTÉTICO")

# Imprimir la información extraída
for item in data:
    print(item)


In [ ]:
import pdfplumber

def is_uppercase(string):
    return string.isupper()

def extract_column_with_uppercase_keyword(pdf_path, keyword):
    with pdfplumber.open(pdf_path) as pdf:
        all_data = []
        capture_next_page = False
        col_index = None
        for page in pdf.pages:
            tables = page.extract_tables()
            for idx, table in enumerate(tables):
                if capture_next_page and col_index is not None:
                    # Si se espera que la tabla continúe desde la página anterior
                    for row in table:
                        all_data.append(row[col_index])
                    capture_next_page = False if idx < len(tables) - 1 else True
                    continue

                for r_idx, row in enumerate(table):
                    uppercase_columns = [i for i, cell in enumerate(row) if is_uppercase(str(cell))]
                    if uppercase_columns:
                        matching_columns = [i for i in uppercase_columns if keyword in str(row[i])]
                        if matching_columns:
                            col_index = matching_columns[0]
                            for subsequent_row in table[r_idx+1:]:
                                all_data.append(subsequent_row[col_index])
                            capture_next_page = True if idx == len(tables) - 1 else False
                            break
            #print("Encabezado detectado en la página:", page.page_number)  # Imprimir número de página para revisión

        return all_data

# Extraer información de la columna con encabezado en mayúsculas "PROGRAMA SINTÉTICO"
data = extract_column_with_uppercase_keyword(pdf_path, "PROGRAMA SINTÉTICO")

# Imprimir la información extraída
for item in data:
    print(item)


In [ ]:
from pdfminer.high_level import extract_text
from unidecode import unidecode

def visualize_keyword_context(pdf_path):
    text = extract_text(pdf_path)
    lines = text.split('\n')

    keywords = ["PROGRAMA SINTETICO", "UNIDADES", "TEMAS"]

    for i, line in enumerate(lines):
        line_without_accent = unidecode(line)

        if any(keyword in line_without_accent for keyword in keywords):
            # Imprimir 5 líneas antes y 5 después de la línea con la palabra clave
            start = max(0, i - 5)
            end = min(len(lines), i + 6)

            for j in range(start, end):
                print(lines[j])
            print('-'*50)  # Separador para cada bloque

visualize_keyword_context(pdf_path)


In [ ]:
from pdfminer.high_level import extract_text
from unidecode import unidecode

def extract_section_with_title(pdf_path):
    text = extract_text(pdf_path)
    lines = text.split('\n')

    # Lista de palabras clave
    keywords = ["PROGRAMA SINTETICO", "UNIDADES", "TEMAS"]

    section_content = []
    capture_content = False
    column_index = None

    for line in lines:
        line_without_accent = unidecode(line)

        # Ignora líneas vacías (saltos de página)
        if not line.strip():
            continue

        if not capture_content:
            parts = line_without_accent.split("  ")
            for keyword in keywords:
                if keyword in parts:
                    column_index = parts.index(keyword)
                    capture_content = True
                    keywords.remove(keyword)  # Eliminamos la palabra clave encontrada de la lista
                    break

        elif capture_content and column_index is not None:
            parts = line_without_accent.split("  ")
            if len(parts) > column_index:
                section_content.append(parts[column_index].strip())
            else:
                # Si la estructura de la tabla parece haber terminado, termina la captura
                break

    return section_content

section = extract_section_with_title(pdf_path)
for line in section:
    print(line)
